https://github.com/woven-planet/l5kit/blob/master/examples/simulation/train.ipynb

## Prepare Data path and load cfg

By setting the `L5KIT_DATA_FOLDER` variable, we can point the script to the folder where the data lies.

Then, we load our config file with relative paths and other configurations (rasteriser, training params...).

### Setup

In [1]:
from pathlib import Path
import os

In [2]:
#NOTE: DONT USE RELATIVE PATHS FOR THE MODELS PROVIDED BY L5
experiments_directory = Path(Path(os.path.abspath('')).parent.parent, "Experiments")
experiments_directory.mkdir(parents=True, exist_ok=True)

data_directory = Path(experiments_directory, "data")
data_directory.mkdir(parents=True, exist_ok=True)

prediction_directory = Path(experiments_directory, "simulation")
prediction_directory.mkdir(parents=True, exist_ok=True)

prediction_training_directory = Path(prediction_directory, "training")
prediction_training_directory.mkdir(parents=True, exist_ok=True)

save_directory = Path(prediction_training_directory, "saved_outputs")
save_directory.mkdir(parents=True, exist_ok=True)

In [3]:
import os
os.chdir(prediction_training_directory)

In [4]:
%%writefile requirements.txt
l5kit
pyyaml
ray==2.0.0rc1
ray[air]
wandb
optuna

Overwriting requirements.txt


In [5]:
%%capture
# !pip install -r requirements.txt
!pip install l5kit pyyaml
!pip install wandb --upgrade
!pip install ray==2.0.0rc1
!pip install "ray[air]"
!pip install optuna

In [6]:
from typing import Dict

from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet50
from tqdm import tqdm

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace, rmse, prob_true_mode, average_displacement_error_oracle, average_displacement_error_mean, final_displacement_error_oracle, final_displacement_error_mean, detect_collision, distance_to_reference_trajectory
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

import os

In [7]:
from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset
from l5kit.rasterization import build_rasterizer
from l5kit.geometry import transform_points
from l5kit.visualization import TARGET_POINTS_COLOR, draw_trajectory
from l5kit.planning.rasterized.model import RasterizedPlanningModel

import os

### Get Data from Wandb

In [8]:
import wandb
wandb.login()

wandb: Currently logged in as: a-sh0ts. Use `wandb login --relogin` to force relogin


True

In [9]:
# Run information
wandb_entity = "l5-demo"
project_name = "l5-simulation"
run_name = "download-l5-data"
run_type = "download"
run_description = """
Download data for the task of training a prediction model
"""
tags = ["download", "data"]

In [10]:
#🪄🐝
run = wandb.init(
    entity=wandb_entity,
    project=project_name,
    job_type=run_type,
    name=run_name,
    notes=run_description,
    tags=tags
)

wandb: Currently logged in as: a-sh0ts (l5-demo). Use `wandb login --relogin` to force relogin


In [11]:
artifact_entity = "l5-demo"
artifact_project = "l5-common"
artifact_name = "l5-data"
artifact_alias = "latest"
artifact_type = "dataset"

In [12]:
#🪄🐝
artifact = run.use_artifact(f"{artifact_entity}/{artifact_project}/{artifact_name}:{artifact_alias}", type=artifact_type)

In [13]:
_ = artifact.download(data_directory)

wandb: Downloading large artifact l5-data:latest, 2386.92MB. 517 files... Done. 0:0:0.1


In [14]:
#BUG: need to seperate runs into download and training due to issues with routing runs after ray.tune
run.finish()

In [15]:
# Dataset is assumed to be on the folder specified
# in the L5KIT_DATA_FOLDER environment variable

# get config
cfg = load_config_data(Path(data_directory, "configurations", "simulation_config.yaml"))

##TODO-Fix with logic flow to select dataset###
cfg["train_data_loader"]["key"] = "scenes/sample.zarr"
cfg["val_data_loader"]["key"] = "scenes/sample.zarr"

l5_data_location = Path(data_directory, "dataset")
# run.config.update(cfg)

In [16]:
# cfg["zarr_dataset_location"] = l5_data_location
os.environ["L5KIT_DATA_FOLDER"] = str(l5_data_location)

## Model

Our baseline is a simple `resnet50` pretrained on `imagenet`. We must replace the input and the final layer to address our requirements.

In [17]:
def build_model(cfg: Dict, num_input_channels, criterion):
    model = RasterizedPlanningModel(
        model_arch=cfg["model_params"]["model_architecture"],
        num_input_channels=num_input_channels,
        num_targets=3 * cfg["model_params"]["future_num_frames"],  # X, Y, Yaw * number of future states,
        weights_scaling= [1., 1., 1.],
        criterion=criterion
        )

    return model

In [18]:
def forward(data, model, criterion):    
    outputs = model(data)
    loss = outputs["loss"]
    return loss, outputs

In [19]:
def train_prediction_model_epoch(data, model, criterion, optimizer):
    loss, outputs = forward(data, model, criterion)
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss, outputs

Our data pipeline map a raw `.zarr` folder into a multi-processing instance ready for training by:
- loading the `zarr` into a `ChunkedDataset` object. This object has a reference to the different arrays into the zarr (e.g. agents and traffic lights);
- wrapping the `ChunkedDataset` into an `AgentDataset`, which inherits from torch `Dataset` class;
- passing the `AgentDataset` into a torch `DataLoader`

# Training

note: if you're on MacOS and using `py_satellite` rasterizer, you may need to disable opencv multiprocessing by adding:
`cv2.setNumThreads(0)` before the following cell. This seems to only affect running in python notebook and it's caused by the `cv2.warpaffine` function

In [20]:
import ray.train as train
from ray.air import session, Checkpoint

In [21]:
from ray import tune
from ray.tune.tuner import Tuner

In [22]:
def train_simulation_model(tuner_cfg : Dict):
    trial_name = session.get_trial_name()
    trial_id = session.get_trial_id()
    trial_readable_name = f"{trial_name}_{trial_id}"
    
    dm = LocalDataManager()
    
    # ==== Configurations
    shuffle = tuner_cfg["shuffle"]
    batch_size = int(tuner_cfg["batch_size"])
    num_workers = tuner_cfg["num_workers"]
    lr = tuner_cfg["lr"]
    max_num_steps = int(tuner_cfg["max_num_steps"])
    dataset_key = tuner_cfg["dataset_key"]
    cfg = tuner_cfg["cfg"]
    
    # ==== Loading Dataset
    rasterizer = build_rasterizer(cfg, dm)

    train_zarr = ChunkedDataset(dm.require(dataset_key)).open()
    train_dataset = AgentDataset(cfg, train_zarr, rasterizer)

    batch_size_per_worker = batch_size // session.get_world_size()
    train_dataloader = DataLoader(train_dataset, shuffle=shuffle, batch_size=batch_size_per_worker, num_workers=num_workers)
    train_dataloader = train.torch.prepare_data_loader(train_dataloader)
    
    # ==== Init model
    criterion = nn.MSELoss(reduction="none")
    num_input_channels = rasterizer.num_channels()
    
    model = build_model(cfg, num_input_channels, criterion)
    model = train.torch.prepare_model(model)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # ==== TRAIN LOOP
    tr_it = iter(train_dataloader)
    progress_bar = range(max_num_steps)
    num_checkpoints = 5
    steps_before_checkpointing = max_num_steps // num_checkpoints
    losses_train = []
    checkpoint_counter = 0
    
    for step in progress_bar:
        try:
            data = next(tr_it)
        except StopIteration:
            tr_it = iter(train_dataloader)
            data = next(tr_it)
            
        model.train()
        torch.set_grad_enabled(True)
        loss, _ = train_prediction_model_epoch(data, model, criterion, optimizer)
        losses_train.append(loss.item())
        avg_loss = np.mean(losses_train)
        metrics = {
            "loss": loss.item(),
            "avg_loss": avg_loss
        }
        
        if train.world_rank() == 0:
            print(metrics)
        
        if (step%steps_before_checkpointing==0) or (step==max_num_steps-1):
            session.report(
                metrics=metrics,
                checkpoint=Checkpoint.from_dict(dict(step=step, model=model)))
            checkpoint_counter += 1
        else:
            session.report(
                metrics=metrics
            )

### Distributed Training using Ray

We calculate the available hardware for our current training sessions and efficiently split CPUs based on GPUs or split CPUs evenly if possible

In [23]:
from ray.train.torch import TorchTrainer
from ray.air.config import RunConfig, ScalingConfig
from ray.air.callbacks.wandb import WandbLoggerCallback #🪄🐝

In [24]:
import multiprocessing

In [25]:
USE_GPU = torch.cuda.is_available()
NUM_GPUS = torch.cuda.device_count()
NUM_CPUS = multiprocessing.cpu_count()

In [26]:
if USE_GPU:
    num_actors = NUM_GPUS
    num_data_workers = NUM_CPUS // num_actors
else:
    num_data_workers = 4 if NUM_CPUS>=4 else NUM_CPUS
    ideal_num_actors = NUM_CPUS // num_data_workers
    num_actors = ideal_num_actors if ideal_num_actors else 1

To use Ray all we need to simply do is wrap the training function above. The only addition needed above was calls to `report.session` to log metrics during training

In [27]:
#NOTE: To figure out if scaling config intuiutin is correct: num_actors divide resources between each actor and within the train func each actor can the utilize the shared resources
trainer = TorchTrainer(
    train_loop_per_worker=train_simulation_model,
    scaling_config=ScalingConfig(num_workers=num_actors, use_gpu=USE_GPU),
)

2022-09-07 21:06:04,412	INFO worker.py:1487 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265.


### Distributed Hyperparemeter Tuning using Ray

Due to Ray's easy interface we can simply extend our normal trainer to Ray's tuner which will allow us to do efficient hyperparameter optimization. In our case we use `optuna`

In [28]:
tuner_train_config = {}
##static
tuner_train_config["shuffle"] = cfg["train_data_loader"]["shuffle"]
tuner_train_config["num_workers"] = num_data_workers
tuner_train_config["dataset_key"] = cfg["train_data_loader"]["key"]

##tunable
tuner_train_config["max_num_steps"] = 5
# tuner_train_config["max_num_steps"] = tune.quniform(1000, 5000, 250)
tuner_train_config["lr"] = tune.loguniform(1e-4, 1e-2)
tuner_train_config["batch_size"] = tune.quniform(6, 24, 6)
cfg["raster_params"]["map_type"] = tune.choice(["py_semantic", "py_satellite"])

tuner_train_config["cfg"] = cfg

In [29]:
from ray.tune.logger import LoggerCallback
from typing import Dict, List

In [30]:
from ray.tune.stopper import ExperimentPlateauStopper
from ray.tune.search.optuna import OptunaSearch

In [31]:
n_search_attempts = 25

In [32]:
optuna_search = OptunaSearch()

In [33]:
tuner = Tuner(
        trainer,
        tune_config=tune.TuneConfig(
            metric="avg_loss", #loss or avg_loss here?
            mode="min",
            search_alg=optuna_search,
            num_samples=n_search_attempts,
        ),
        param_space={
            "train_loop_config": tuner_train_config
        },
        run_config=RunConfig(
            stop=ExperimentPlateauStopper("avg_loss"),
            callbacks=[WandbLoggerCallback(project=f"{project_name}-trials", save_checkpoints=True),]))  #🪄🐝

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Executing `.fit()` may leave less than 20% of CPUs in this cluster for Dataset execution, which can lead to resource contention or hangs. To avoid this, reserve at least 20% of node CPUs for Dataset execution by setting `_max_cpu_fraction_per_node = 0.8` in the Trainer scaling_config. See https://docs.ray.io/en/master/data/dataset-internals.html#datasets-and-tune for more info.
  


### Aggregrate and Report Metrics from All Trials

In [34]:
analysis = tuner.fit()

[I 2022-09-07 21:06:06,636] A new study created in memory with name: optuna
2022-09-07 21:06:06,654	INFO wandb.py:119 -- Already logged into W&B.


Trial name,status,loc,train_loop_config...,train_loop_config...,train_loop_config/lr,iter,total time (s),loss,avg_loss,_timestamp
TorchTrainer_e3c5f5d4,TERMINATED,10.150.0.3:25992,12,py_satellite,0.0072049,5,60.1556,74.5676,77.8439,1662584829
TorchTrainer_e5c1813c,TERMINATED,10.150.0.3:26064,18,py_semantic,0.000556754,5,87.8817,67.7284,90.8705,1662584862
TorchTrainer_e5c9abaa,TERMINATED,10.150.0.3:27061,12,py_semantic,0.000177311,5,59.6294,17.9698,35.8953,1662584903
TorchTrainer_0f7cd6a2,TERMINATED,10.150.0.3:27382,6,py_satellite,0.00160407,5,37.5158,5.84599,98.9697,1662584913
TorchTrainer_2328e218,TERMINATED,10.150.0.3:28334,6,py_semantic,0.000690849,5,40.6134,138.552,41.8565,1662584968
TorchTrainer_4160e4a6,TERMINATED,10.150.0.3:28343,18,py_satellite,0.000392097,5,84.8762,47.9169,59.9695,1662585013
TorchTrainer_41bc41c0,TERMINATED,10.150.0.3:29336,18,py_satellite,0.00249845,5,82.1401,14.7764,42.0436,1662585061
TorchTrainer_611d7426,TERMINATED,10.150.0.3:30153,18,py_semantic,0.00158679,5,81.7708,93.3388,68.7019,1662585114
TorchTrainer_7e39ebde,TERMINATED,10.150.0.3:30991,24,py_semantic,0.000487912,5,105.296,37.8674,52.5062,1662585183
TorchTrainer_9b8feb8e,TERMINATED,10.150.0.3:31815,12,py_satellite,0.000313175,5,55.4984,119.398,69.8637,1662585186


(RayTrainWorker pid=26063) 2022-09-07 21:06:13,828	INFO config.py:72 -- Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=26176) 2022-09-07 21:06:18,306	INFO config.py:72 -- Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=26063) /opt/conda/lib/python3.7/site-packages/l5kit/planning/rasterized/model.py:31: UserWarning: There is no pre-trained model with num_in_channels != 3, first layer will be reset
(RayTrainWorker pid=26063)   warnings.warn("There is no pre-trained model with num_in_channels != 3, first layer will be reset")
(RayTrainWorker pid=26063) /opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
(RayTrainWorker pid=26063)   f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
(RayTrainWorker pid=26063) /opt/conda/lib/python3.7/site-pac

(RayTrainWorker pid=26063) {'loss': 34.04572677612305, 'avg_loss': 34.04572677612305}


wandb: Adding directory to artifact (/home/jupyter/ray_results/TorchTrainer_2022-09-07_21-06-06/TorchTrainer_e3c5f5d4_1_batch_size=12.0000,format_version=4,future_num_frames=50,history_num_frames=5,model_architecture=resnet50,_2022-09-07_21-06-06/checkpoint_000000)... 

...

Result for TorchTrainer_f8df3500:
  _time_this_iter_s: 4.393399715423584
  _timestamp: 1662585755
  _training_iteration: 5
  avg_loss: 79.0663631439209
  date: 2022-09-07_21-22-35
  done: false
  experiment_id: 0454cfca86fa4aebb8278f28870bee72
  hostname: anish-l5-kit
  iterations_since_restore: 5
  loss: 121.65872955322266
  node_ip: 10.150.0.3
  pid: 9097
  should_checkpoint: true
  time_since_restore: 31.116479635238647
  time_this_iter_s: 4.407530307769775
  time_total_s: 31.116479635238647
  timestamp: 1662585755
  timesteps_since_restore: 0
  training_iteration: 5
  trial_id: f8df3500
  warmup_time: 0.004829883575439453
  


Done. 1.0s
wandb: Adding directory to artifact (/home/jupyter/ray_results/TorchTrainer_2022-09-07_21-06-06/TorchTrainer_f8df3500_25_batch_size=6.0000,format_version=4,future_num_frames=50,history_num_frames=5,model_architecture=resnet50,_2022-09-07_21-22-00/checkpoint_000004)... Done. 0.2s


avg_loss,█▄▁▁▃
iterations_since_restore,▁▃▅▆█
loss,█▄▁▃█
time_since_restore,▁▃▅▆█
time_this_iter_s,█▁▁▁▁
time_total_s,▁▃▅▆█
timestamp,▁▃▅▆█
timesteps_since_restore,▁▁▁▁▁
training_iteration,▁▃▅▆█
warmup_time,▁▁▁▁▁
avg_loss,79.06636


Result for TorchTrainer_f8df3500:
  _time_this_iter_s: 4.393399715423584
  _timestamp: 1662585755
  _training_iteration: 5
  avg_loss: 79.0663631439209
  date: 2022-09-07_21-22-35
  done: true
  experiment_id: 0454cfca86fa4aebb8278f28870bee72
  experiment_tag: 25_batch_size=6.0000,format_version=4,future_num_frames=50,history_num_frames=5,model_architecture=resnet50,render_ego_history=False,step_time=0.1000,dataset_meta_key=meta_json,disable_traffic_light_faces=False,ego_center=0_25_0_5,filter_agents_threshold=0.5000,map_type=py_semantic,pixel_size=0_5_0_5,raster_size=224_224,satellite_map_key=aerial_map_aerial_map_png,semantic_map_key=semantic_map_semantic_map_pb,set_origin_to_bottom=True,batch_size=12,key=scenes_sample_zarr,num_workers=16,perturb_probability=0.0000,shuffle=True,checkpoint_every_n_steps=10000,eval_every_n_steps=10000,max_num_steps=5,batch_size=12,key=scenes_sample_zarr,num_workers=16,shuffle=False,dataset_key=scenes_sample_zarr,lr=0.0051,max_num_steps=5,num_workers=4,

2022-09-07 21:22:44,584	INFO tune.py:759 -- Total run time: 997.96 seconds (997.79 seconds for the tuning loop).


In [35]:
import time

In [36]:
time.sleep(30)

In [37]:
analysis_df = analysis.get_dataframe()

In [38]:
analysis_df

,loss,avg_loss,_timestamp,_time_this_iter_s,_training_iteration,time_this_iter_s,should_checkpoint,done,timesteps_total,episodes_total,...,config/train_loop_config/cfg/train_data_loader/perturb_probability,config/train_loop_config/cfg/train_data_loader/shuffle,config/train_loop_config/cfg/train_params/checkpoint_every_n_steps,config/train_loop_config/cfg/train_params/eval_every_n_steps,config/train_loop_config/cfg/train_params/max_num_steps,config/train_loop_config/cfg/val_data_loader/batch_size,config/train_loop_config/cfg/val_data_loader/key,config/train_loop_config/cfg/val_data_loader/num_workers,config/train_loop_config/cfg/val_data_loader/shuffle,logdir
0,74.567551,77.843877,1662584829,9.905044,5,9.896055,True,False,NaN,NaN,...,0.0,True,10000,10000,5,12,scenes/sample.zarr,16,False,/home/jupyter/ray_results/TorchTrainer_2022-09...
1,67.728432,90.870517,1662584862,16.227512,5,16.154536,True,False,NaN,NaN,...,0.0,True,10000,10000,5,12,scenes/sample.zarr,16,False,/home/jupyter/ray_results/TorchTrainer_2022-09...
2,17.969776,35.895309,1662584903,9.094891,5,9.252898,True,False,NaN,NaN,...,0.0,True,10000,10000,5,12,scenes/sample.zarr,16,False,/home/jupyter/ray_results/TorchTrainer_2022-09...
3,5.845992,98.969687,1662584913,5.566701,5,5.570402,True,False,NaN,NaN,...,0.0,True,10000,10000,5,12,scenes/sample.zarr,16,False,/home/jupyter/ray_results/TorchTrainer_2022-09...
4,138.551712,41.856454,1662584968,4.608834,5,4.618596,True,False,NaN,NaN,...,0.0,True,10000,10000,5,12,scenes/sample.zarr,16,False,/home/jupyter/ray_results/TorchTrainer_2022-09...
5,47.916885,59.969492,1662585013,14.004685,5,13.904366,True,False,NaN,NaN,...,0.0,True,10000,10000,5,12,scenes/sample.zarr,16,False,/home/jupyter/ray_results/TorchTrainer_2022-09...
6,14.776404,42.043575,1662585061,12.540035,5,13.211249,True,False,NaN,NaN,...,0.0,True,10000,10000,5,12,scenes/sample.zarr,16,False,/home/jupyter/ray_results/TorchTrainer_2022-09...
7,93.338753,68.701921,1662585114,12.791445,5,12.706946,True,False,NaN,NaN,...,0.0,True,10000,10000,5,12,scenes/sample.zarr,16,False,/home/jupyter/ray_results/TorchTrainer_2022-09...
8,37.867397,52.506245,1662585183,18.638955,5,18.685607,True,False,NaN,NaN,...,0.0,True,10000,10000,5,12,scenes/sample.zarr,16,False,/home/jupyter/ray_results/TorchTrainer_2022-09...
9,119.398010,69.863711,1662585186,9.733388,5,9.714789,True,False,NaN,NaN,...,0.0,True,10000,10000,5,12,scenes/sample.zarr,16,False,/home/jupyter/ray_results/TorchTrainer_2022-09...


In [39]:
# Run information
wandb_entity = "l5-demo"
project_name = "l5-simulation"
run_name = "train-simulation-model"
run_type = "train"
run_description = """
Train simulation model
"""
tags = ["train", "simulation"]

In [40]:
#🪄🐝
run = wandb.init(
    entity=wandb_entity,
    project=project_name,
    job_type=run_type,
    name=run_name,
    notes=run_description,
    tags=tags,
    config=cfg
)

In [41]:
#BUG: to force a connection on the lineage graph
#🪄🐝
artifact = run.use_artifact(f"{artifact_entity}/{artifact_project}/{artifact_name}:{artifact_alias}", type=artifact_type)

In [42]:
#🪄🐝
analysis_table = wandb.Table(dataframe=analysis_df)

In [43]:
#BUG: run gets lost after tune job due to change in cwd. Forced to make 2 runs
if len(analysis_table.data) == 0:
    raise ValueError("bad table for some reason")
else:
    run.log({"analysis_table": analysis_table})
    run.finish()